In [1]:
import pandas as pd
import geopandas as gpd
import pickle
import numpy as np
import requests

In [2]:
survey_path = "../../../results/surveys/egt_2010/cleaned/trips.parquet"
spatial_path = "../../../results/surveys/egt_2010/spatial.parquet"

calibration_path = "../../../results/road/freeflow/calibration_cache_api.pickle"
congestion_path = "../../../results/road/congestion_factors.parquet"

routing_endpoint = "http://localhost:8054/router/road"
departure_time = 4 * 3600
maximum_batch_size = 400

output_path = "../../../results/road/routing.parquet"

In [3]:
# Load survey data
df_survey = pd.read_parquet(survey_path)
df_spatial = gpd.read_parquet(spatial_path)

In [4]:
# Prepare spatial data
df_spatial["origin_x"] = df_spatial["origin_geometry"].x
df_spatial["origin_y"] = df_spatial["origin_geometry"].y
df_spatial["destination_x"] = df_spatial["destination_geometry"].x
df_spatial["destination_y"] = df_spatial["destination_geometry"].y

In [5]:
# Merge in spatial data
df_survey = pd.merge(df_survey, df_spatial)[[
    "trip_id",
    "origin_x", "origin_y",
    "destination_x", "destination_y",
    "departure_time"
]].copy()

In [6]:
# Load freeflow routing parameters
with open(calibration_path, "rb") as f:
    history = pickle.load(f)

objective = np.inf
best = None

for item in history:
    if item["objective"] < objective:
        best = item
        # break # use first

settings = best["settings"]
settings

{'major_factor': 1.0000000000964675,
 'intermediate_factor': 1.000000000110372,
 'minor_factor': 1.0000000000071958,
 'major_crossing_penalty_s': 4.20563605332865,
 'minor_crossing_penalty_s': 1.1414043230895694e-09}

In [7]:
# Prepare requests
df_survey["request_index"] = np.arange(len(df_survey))

# Convert to requests
request_list = []

for index, row in df_survey.iterrows():
    request_list.append({
        "request_index": int(row["request_index"]),
        "origin_x": row["origin_x"],
        "origin_y": row["origin_y"],
        "destination_x": row["destination_x"],
        "destination_y": row["destination_y"],
        "departure_time_s": departure_time
    })

In [8]:
# Prepare querying
def query_requests(request_list, settings):
    df_response = []
    batch_index = 0

    while batch_index * maximum_batch_size < len(request_list):
        batch = request_list[batch_index * maximum_batch_size : (batch_index + 1) * maximum_batch_size]

        response = requests.post(routing_endpoint, json = {
            "batch": batch,
            "freespeed": settings
        })

        df_response.append(pd.DataFrame.from_records(response.json()))
        batch_index += 1

    return pd.concat(df_response)

In [9]:
# Test connection
assert len(query_requests(request_list[:5], settings)) == 5

In [10]:
# Obtain response
df_response = query_requests(request_list, settings)

In [11]:
df_response = pd.merge(df_response, df_survey[[
    "request_index", "trip_id", "departure_time"
]], on = "request_index")[[
    "trip_id", "departure_time",
    "in_vehicle_distance_km", "in_vehicle_time_min",
    "access_time_min", "egress_time_min",
    "access_distance_km", "egress_distance_km"
]]

In [12]:
# Remove NaN
df_response = df_response[~df_response["departure_time"].isna()]

# Calculate hour
df_response["hour"] = df_response["departure_time"] // 3600
df_response.loc[df_response["hour"] > 23, "hour"] -= 24
df_response["hour"] = df_response["hour"].astype(int)

In [13]:
df_congestion = pd.read_parquet(congestion_path)
df_response = pd.merge(df_response, df_congestion, on = "hour")
df_response["in_vehicle_time_min"] *= df_response["congestion_factor"]

In [14]:
df_response.to_parquet(output_path)